# Part V. Semi-supervised Classification on  Cardiotocography Data Set

In [30]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA, QuadraticDiscriminantAnalysis as QDA
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix,accuracy_score

In [21]:
train_data_labeled = pd.read_csv("../data/CTG/CTG_Train_labeled.csv").dropna()
train_data_unlabeled = pd.read_csv("../data/CTG/CTG_Train_unlabeled.csv").dropna()
test_data = pd.read_csv("../data/CTG/CTG_Test.csv").dropna()
Xtest = test_data.loc[:,:"Tendency"]
ytest = test_data.NSP

## 1. Try three classification methods 
using only labelled training data, and test their performances on 
the test data. Please try to optimize the tuning parameters as best as you can.  Report the results 
and conclusions.

In [22]:
X = train_data_labeled.loc[:,:"Tendency"]
y = train_data_labeled.NSP

In [26]:
# 1 LDA
lda = LDA()
lda.fit(X,y)

print("classification accuracy:", lda.score(Xtest,ytest))
print("mis-classification error:",1-lda.score(Xtest,ytest))
print("confusion matrix:\n",confusion_matrix(lda.predict(Xtest),ytest))

classification accuracy: 0.8937875751503006
mis-classification error: 0.1062124248496994
confusion matrix:
 [[366  25   4]
 [ 11  46   6]
 [  3   4  34]]
